# transcriptome assembly and annotation

Aaron Lee, June 2023

This data analysis pipeline largely follows the workflow outlined by Ya Yang and Diego Morales-Briones for transcriptome assembly in phylogenomic dataset/orthology analysis https://bitbucket.org/yanglab/phylogenomic_dataset_construction/src/master/. It is slightly less conservative, with fewer raw data cleaning steps. The workflow is as follows:
1. quality checking
2. quality trimming
3. deduplication
4. removing organellar reads
5. de novo transcriptome assembly
6. process transcriptome sequences
7. coding sequence annotation

## dependencies
Anaconda, or any of the derivatives (eg miniconda, mamba, etc). All of the following programs will be wrapped up in a conda environment. Additionally, you will need the chimera removal script from the phylogenomic dataset construction pipeline.

- FastQC
- TRIMMOMATIC
- PRINSEQ
- Bowtie2
- transrate
- Trinity
- BLAST+
- transdecoder
- chimera removal script

The following code will help us to write batch scripts for the SLURM job scheduler. You will need to go to your command line to send the batch scripts using the `sbatch job-script.sh` command.

In [2]:
import subprocess, sys, os

# set working directory
wd = os.path.abspath("/home/yangya/lee02893/sceletium_nox")

# make working directory
if not os.path.exists(wd):
    subprocess.run(["mkdir", wd])

# set email
email = "lee02893@umn.edu"

# set reads
reads1 = "/home/yangya/lee02893/sceletium/data/reduced_data/Sceletium_nova_S25_R1.reduced.fastq" #reduced R1
reads2 = "/home/yangya/lee02893/sceletium/data/reduced_data/Sceletium_nova_S25_R2.reduced.fastq" #reduced R2
#reads1 = "/home/yangya/lee02893/sceletium/data/Sceletium_nova_S25_R1_001.fastq.gz" #complete R1
#reads2 = "/home/yangya/lee02893/sceletium/data/Sceletium_nova_S25_R2_001.fastq.gz" #complete R2


# 1. quality checking

In [9]:
with open(os.path.join(wd, "fastqc.sh"), "w+") as outf:
    outf.write("#!/usr/bin/bash\n")
    outf.write("#SBATCH --mail-user={}\n").format(email)
    outf.write("#SBATCH --mail-type=ALL\n")
    outf.write("#SBATCH --job-name=fastqc\n")
    outf.write("#SBATCH --time=1:00:00\n")
    outf.write("#SBATCH --partition=msismall\n")
    outf.write("#SBATCH --ntasks=16\n")
    outf.write("#SBATCH --mem=8g\n\n")
    
    #outf.write("module load fastqc\n")
    outf.write("source /home/yangya/lee02893/.bashrc\n")
    outf.write("source activate transcriptome\n")
    outf.write("cd {}\n".format(wd))

    outf.write("fastqc -t 16 {} {}".format(reads1, reads2))

CompletedProcess(args=['sbatch', 'fastqc.sh'], returncode=0)

# 2. quality trimming

In [13]:
with open(os.path.join(wd, "trimmomatic.sh"), "w+") as outf:
    outf.write("#!/usr/bin/bash\n")
    outf.write("#SBATCH --mail-user={}\n".format(email)
    outf.write("#SBATCH --mail-type=ALL\n")
    outf.write("#SBATCH --job-name=trim\n")
    outf.write("#SBATCH --time=12:00:00\n")
    outf.write("#SBATCH --partition=msismall\n")
    outf.write("#SBATCH --ntasks=16\n")
    outf.write("#SBATCH --mem=8g\n")

    outf.write("source /home/yangya/lee02893/.bashrc\n")
    outf.write("source activate transcriptome\n")
    outf.write("cd {}\n".format(wd))
    outf.write("trimmomatic PE -threads 16 {} {} Sceletium_nova.trim.1P.fastq Sceletium_nova.trim.1U.fastq Sceletium_nova.trim.2P.fastq Sceletium_nova.trim.2U.fastq ILLUMINACLIP:/panfs/roc/msisoft/trimmomatic/0.33//adapters/all_illumina_adapters.fa:2:30:7 SLIDINGWINDOW:4:20 LEADING:20 TRAILING:20 MINLEN:35".format(reads1, reads2))


# 3. deduplication

In [16]:
with open(os.path.join(wd, "prinseq.sh"), "w+") as outf:
    outf.write("#!/usr/bin/bash\n")
    outf.write("#SBATCH --mail-user={}\n").format(email)
    outf.write("#SBATCH --mail-type=ALL\n")
    outf.write("#SBATCH --job-name=prinseq\n")
    outf.write("#SBATCH --time=12:00:00\n")
    outf.write("#SBATCH --partition=msismall\n")
    outf.write("#SBATCH --ntasks=16\n")
    outf.write("#SBATCH --mem=8g\n")

    outf.write("source /home/yangya/lee02893/.bashrc\n")
    outf.write("source activate transcriptome\n")
    outf.write("cd {}\n".format(wd))
    outf.write("prinseq-lite.pl -verbose -fastq Sceletium_nova.trim.1P.fastq -fastq2 Sceletium_nova.trim.2P.fastq -derep 123 -out_good Sceletium_nova_dedup -out_bad null")

# 4. removing organellar reads

In [20]:
with open(os.path.join(wd, "bowtie2.sh"), "w+") as outf:
    outf.write("#!/usr/bin/bash\n")
    outf.write("#SBATCH --mail-user={}\n").format(email)
    outf.write("#SBATCH --mail-type=ALL\n")
    outf.write("#SBATCH --job-name=bowtie2\n")
    outf.write("#SBATCH --time=12:00:00\n")
    outf.write("#SBATCH --partition=msismall\n")
    outf.write("#SBATCH --ntasks=16\n")
    outf.write("#SBATCH --mem=8g\n")
    
    outf.write("source /home/yangya/lee02893/.bashrc\n")
    outf.write("source activate transcriptome\n")
    outf.write("cd {}\n".format(wd))
    
    # build bowtie2 index for ice plant chloroplast
    outf.write("bowtie2-build /home/yangya/lee02893/sceletium/Mesembryanthemum_crystallinum.cp.fa Mcry_cp\n")
    
    # align reads to ice plant chloroplast and retain reads that DO NOT map (--un-conc)
    outf.write("bowtie2 -x Mcry_cp -1 Sceletium_nova_dedup_1.fastq -2 Sceletium_nova_dedup_2.fastq -S Sceletium_cp.sam --un-conc Sceletium_bt2_unaligned_%.fastq\n")
    # remove large SAM file
    outf.write("rm *.sam")


# 5. de novo transcriptome assembly

In [23]:
with open(os.path.join(wd, "trinity.sh"), "w+") as outf:
    outf.write("#!/usr/bin/bash\n")
    outf.write("#SBATCH --mail-user={}\n").format(email)
    outf.write("#SBATCH --mail-type=ALL\n")
    outf.write("#SBATCH --job-name=trinity\n")
    outf.write("#SBATCH --time=1-00:00:00\n")
    outf.write("#SBATCH --partition=msismall\n")
    outf.write("#SBATCH --ntasks=16\n")
    outf.write("#SBATCH --mem=200g\n")
    
    outf.write("source /home/yangya/lee02893/.bashrc\n")
    outf.write("source activate transcriptome\n")
    outf.write("cd {}\n".format(wd))
    
    outf.write("module load java")
    outf.write("Trinity --seqType fq --max_memory 200G --CPU 16 --verbose --left Sceletium_bt2_unaligned_1.fastq --right Sceletium_bt2_unaligned_2.fastq --output trinity_Sceletium_nova --full_cleanup")

# 6. process transcriptome sequences

In [ ]:
with open(os.path.join(wd, "process_transcripts.sh"), "w+") as outf:
    outf.write("#!/usr/bin/bash\n")
    outf.write("#SBATCH --mail-user={}}\n").format(email)
    outf.write("#SBATCH --mail-type=ALL\n")
    outf.write("#SBATCH --job-name=proc_seqs\n")
    outf.write("#SBATCH --time=1-00:00:00\n")
    outf.write("#SBATCH --partition=msismall\n")
    outf.write("#SBATCH --ntasks=20\n")
    outf.write("#SBATCH --mem=40g\n")
    #outf.write("module load ncbi_blast+\n")
    
    outf.write("source /home/yangya/lee02893/.bashrc\n")
    outf.write("source activate transcriptome\n")
    outf.write("cd {}\n".format(wd))
    
    # blastx search Trinity transcripts against ice plant amino acid sequences
    outf.write("blastx -db /home/yangya/lee02893/blastdb/iceplant_protein -out Sceletium_nova_blastx.tsv -query trinity_Sceletium_nova/Trinity.fasta -num_threads 20 -evalue 10 -outfmt "6 qseqid qlen sseqid slen frames pident nident length mismatch gapopen qstart qend sstart send evalue bitscore" -max_target_seqs 100\n")
    # remove chimeric sequences using Ya's script
    outf.write("python /home/yangya/lee02893/software/phylogenomic_dataset_construction/scripts/detect_chimera_from_blastx_modified.py Sceletium_nova_blastx.tsv ./\n")
    
    # remove poorly supported sequences using transrate
    outf.write("transrate --assembly trinity_Sceletium_nova/Trinity.fasta --left Sceletium_nova.bt2.1P.fq --right Sceletium_nova.bt2.2P.fq")
    
    # call "unigenes"
    # usage: python get_unigenes.py [transrate directory] [chimera directory]
    outf.write("python /home/yangya/lee02893/github/genomics_tools/transcriptome_tools/get_unigenes.py ./ ./")
    outf.write("mv unigenes.Trinity.fasta Sceletium_nova.unigenes.fasta")

# 7. coding sequence annotation

In [ ]:
with open(os.path.join(wd, "transdecoder.sh"), "w+") as outf:
    outf.write("#!/usr/bin/bash\n")
    outf.write("#SBATCH --mail-user={}\n").format(email)
    outf.write("#SBATCH --mail-type=ALL\n")
    outf.write("#SBATCH --job-name=trinity\n")
    outf.write("#SBATCH --time=1-00:00:00\n")
    outf.write("#SBATCH --partition=msismall\n")
    outf.write("#SBATCH --ntasks=20\n")
    outf.write("#SBATCH --mem=40g\n")
    
    outf.write("source /home/yangya/lee02893/.bashrc\n")
    outf.write("source activate transcriptome\n")
    outf.write("cd {}\n".format(wd))
    
    # run TransDecoder, including blastp hits to ice plant
    outf.write("TransDecoder.LongOrfs -t Sceletium_nova.unigenes.fasta\n")
    outf.write("blastp -query Sceletium_nova.unigenes.fasta -db /home/yangya/lee02893/blastdb/iceplant_protein -max_target_seqs 1 -outfmt 6 -evalue 1e-5 -num_threads 8 > Sceletium_nova.longest_orfs.blastp\n")
    outf.write("TransDecoder.Predict -t Sceletium_nova.unigenes.fasta --retain_blastp_hits Sceletium_nova.longest_orfs.blastp\n")